In [44]:
# Import required libraries

import pandas as pd
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from tkinter import Tk, Label, Entry, Button, StringVar, OptionMenu

def train_and_save_model():

    # Load datasets
    exercise_data = pd.read_csv('exercise.csv')
    calorie_data = pd.read_csv('calories.csv')

    # Combine exercise and calorie data
    data = exercise_data.copy()
    data['Calories'] = calorie_data['Calories']

    # Define features and target variable
    features = ['Age', 'Gender', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
    target = 'Calories'
    
    X = data[features]
    y = data[target]

    # Define preprocessing pipeline
    numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
    categorical_features = ['Gender']

    # Preprocessing for numeric data
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    # Create and train the pipeline
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ])

    # Fit the model
    model_pipeline.fit(X, y)

    # Save the trained model
    with open('pipeline.pkl', 'wb') as file:
        pickle.dump(model_pipeline, file)

def load_trained_model(filepath):

    try:
        with open(filepath, 'rb') as file:
            return pickle.load(file)
    except EOFError:
        print("Error: The file is empty or corrupted.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def make_calorie_prediction():

    # Load the model
    model = load_trained_model('pipeline.pkl')
    
    if model is None:
        result_label.config(text="Model not loaded.")
        return
    
    # Get user inputs
    age_years = float(age_entry.get())
    height_in_inches = float(height_entry.get())
    weight_in_pounds = float(weight_entry.get())
    exercise_duration = float(duration_entry.get())
    heart_rate_bpm = float(heart_rate_entry.get())
    body_temp_fahrenheit = float(body_temp_entry.get())
    gender = gender_var.get()  # Gender is either 'male' or 'female'

    # Convert units to metric
    height_cm = height_in_inches * 2.54
    weight_kg = weight_in_pounds * 0.453592
    body_temp_celsius = (body_temp_fahrenheit - 32) * 5 / 9

    # Prepare data for prediction
    input_data = pd.DataFrame({
        'Age': [age_years],
        'Height': [height_cm],
        'Weight': [weight_kg],
        'Duration': [exercise_duration],
        'Heart_Rate': [heart_rate_bpm],
        'Body_Temp': [body_temp_celsius],
        'Gender': [gender]
    })

    # Predict calories burned
    calorie_prediction = model.predict(input_data)
    result_label.config(text=f"Calories Burnt: {calorie_prediction[0]:.2f}")

# GUI Setup
def setup_gui():
  
    # Create main window
    root = Tk()
    root.title("Calorie Burn Prediction")

    # Title label
    Label(root, text="Calorie Burn Prediction", bg="blue", fg="white", font=("Helvetica", 16)).grid(row=0, columnspan=2)

    # Gender dropdown
    Label(root, text="Gender").grid(row=1, column=0)
    global gender_var
    gender_var = StringVar()
    gender_var.set("male")
    OptionMenu(root, gender_var, 'male', 'female').grid(row=1, column=1)

    # Input fields
    Label(root, text="Age (years)").grid(row=2, column=0)
    global age_entry
    age_entry = Entry(root)
    age_entry.grid(row=2, column=1)

    Label(root, text="Height (inches)").grid(row=3, column=0)
    global height_entry
    height_entry = Entry(root)
    height_entry.grid(row=3, column=1)

    Label(root, text="Weight (pounds)").grid(row=4, column=0)
    global weight_entry
    weight_entry = Entry(root)
    weight_entry.grid(row=4, column=1)

    Label(root, text="Duration (minutes)").grid(row=5, column=0)
    global duration_entry
    duration_entry = Entry(root)
    duration_entry.grid(row=5, column=1)

    Label(root, text="Heart Rate (bpm)").grid(row=6, column=0)
    global heart_rate_entry
    heart_rate_entry = Entry(root)
    heart_rate_entry.grid(row=6, column=1)

    Label(root, text="Body Temperature (°F)").grid(row=7, column=0)
    global body_temp_entry
    body_temp_entry = Entry(root)
    body_temp_entry.grid(row=7, column=1)

    # Predict button
    Button(root, text="Predict", command=make_calorie_prediction).grid(row=8, columnspan=2)

    # Result label
    global result_label
    result_label = Label(root, text="", font=("Helvetica", 12))
    result_label.grid(row=9, columnspan=2)

    # Run the GUI main loop
    root.mainloop()

# Run the GUI setup
setup_gui()